In [11]:
import time
import numpy as np

def check(arr):
        repeats = set()
        visited = {(i, j): 0 for i in range(BOARD_SIZE) for j in range(BOARD_SIZE)}
        for i in range(0, BOARD_SIZE - 2):
            for j in range(BOARD_SIZE):
                if visited[(i, j)] == 1 or visited[(j, i)] == 1:
                    continue
                if arr[i, j] != 'nan' and (arr[i+1:i+3, j] == arr[i, j]).all():
                    repeats.add((i+1, j))
                    visited[(i, j)] = 1
                    visited[(i+1, j)] = 1
                    visited[(i+2, j)] = 1
                if arr[j, i] != 'nan' and (arr[j, i+1:i+3] == arr[j, i]).all():
                    repeats.add((j, i+1))
                    visited[(j, i)] = 1
                    visited[(j, i+1)] = 1
                    visited[(j, i+2)] = 1
        return repeats

def check2(arr):
        repeats = set()
        for i in range(BOARD_SIZE):
            row = arr[i]
            row_view = np.lib.stride_tricks.as_strided(row, (BOARD_SIZE - 2, 3), (4, 4))
            connected = np.all(row_view == row_view[:, 0, None], axis=1)
            equals = np.where(connected)[0]
            repeats.update([(i, j + 1) for j in equals])
            
            col = arr[:, i]
            col_view = np.lib.stride_tricks.as_strided(col, (BOARD_SIZE - 2, 3), (24, 24))
            connected = np.all(col_view == col_view[:, 0, None], axis=1)
            equals = np.where(connected)[0]
            repeats.update([(j + 1, i) for j in equals])
        return repeats


In [13]:
def check2(arr):
        repeats = set()
        for i in range(0, BOARD_SIZE - 2):
            for j in range(BOARD_SIZE):
                if arr[i, j] != 'nan' and (arr[i+1:i+3, j] == arr[i, j]).all():
                    repeats.add((i+1, j))
                if arr[j, i] != 'nan' and (arr[j, i+1:i+3] == arr[j, i]).all():
                    repeats.add((j, i+1))
        return repeats


In [17]:
BOARD_SIZE = 6
MAX_TURN = 100
N_ROWS = 1200
np.random.seed(0)
BOARD = np.random.randint(0, 5, (BOARD_SIZE, BOARD_SIZE))
BOARD[0, 1] = 3
BOARD[0, 0] = 3
BOARD[2, 0] = 3
BOARD[2, 2] = 1
print(BOARD)

[[3 3 3 3 3 1]
 [3 2 4 0 0 4]
 [3 1 1 1 1 0]
 [1 4 3 0 3 0]
 [2 3 0 1 3 3]
 [3 0 1 1 1 0]]


In [19]:
row = test[:, 0]
row_view = np.lib.stride_tricks.as_strided(row, (4, 3), (test.itemsize*6, test.itemsize*6))
print(row_view)

[['G' 'G' 'B']
 ['G' 'B' 'B']
 ['B' 'B' 'B']
 ['B' 'B' 'G']]


In [18]:
%timeit check(BOARD)
print(check(BOARD))

180 µs ± 2.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
{(1, 0), (5, 3), (2, 2), (0, 1)}


In [19]:
%timeit check2(BOARD)
print(check2(BOARD))

304 µs ± 29.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
{(0, 1), (5, 3), (0, 3), (2, 3), (0, 2), (2, 2), (1, 0)}


In [9]:
def eliminate(board, func=lambda x: (x - 2) ** 2):
        arr = board
        to_eliminate = np.zeros((6, 6), dtype=int)
        directions = np.array([[1, 0], [-1, 0], [0, 1], [0, -1]])
        to_visit = check(arr)
        score = 0

        for coord in to_visit:
            if to_eliminate[coord[0], coord[1]] == 1:
                continue
            head = 0
            connected = [coord, ]
            while head < len(connected):
                current = connected[head]
                to_eliminate[current[0], current[1]] = 1
                for d in directions:
                    neighbor = current + d
                    if (neighbor < 0).any() or (neighbor >= 6).any():
                        continue
                    if (arr[neighbor[0], neighbor[1]] == arr[current[0], current[1]]
                            and to_eliminate[neighbor[0], neighbor[1]] == 0):
                        connected.append(neighbor)
                head += 1
            score += func(len(connected))

        col_eliminated = np.sum(to_eliminate, axis=1)
        # Return the total score and the number of columns eliminated
        return score, col_eliminated


In [13]:
%timeit eliminate(BOARD)

867 µs ± 49.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [108]:

def eliminate2(board, func=lambda x: (x - 2) ** 2):
        arr = board
        to_eliminate = np.zeros((6, 6), dtype=int)
        directions = np.array([[1, 0], [-1, 0], [0, 1], [0, -1]])
        to_visit = check2(arr)
        score = 0

        for coord in to_visit:
            if to_eliminate[coord[0], coord[1]] == 1:
                continue
            head = 0
            connected = [coord, ]
            while head < len(connected):
                current = connected[head]
                to_eliminate[current[0], current[1]] = 1
                for d in directions:
                    neighbor = current + d
                    if (neighbor < 0).any() or (neighbor >= 6).any():
                        continue
                    if (arr[neighbor[0], neighbor[1]] == arr[current[0], current[1]]
                            and to_eliminate[neighbor[0], neighbor[1]] == 0):
                        connected.append(neighbor)
                head += 1
            score += func(len(connected))

        col_eliminated = np.sum(to_eliminate, axis=1)
        # Return the total score and the number of columns eliminated
        return score, col_eliminated


In [109]:
%timeit eliminate2(BOARD)

1.24 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
